# Nettoyage des Données eBay - Laptops

Ce notebook a pour objectif de charger, nettoyer et transformer les données issues de fichiers CSV contenant des informations sur des laptops vendus sur eBay.
Les principales opérations effectuées sont :

- **Nettoyage de la colonne `Price`** : suppression des caractères inutiles et conversion en type numérique.
- **Uniformisation des unités** pour les colonnes telles que `RAM` et `Storage` (conversion en GB).
- **Extraction et conversion** des valeurs de la colonne `Screen Size`.
- **Imputation des valeurs manquantes** selon des règles définies.
- **Nettoyage des titres** pour retirer les informations non essentielles.
- **Suppression des doublons** en ne conservant que l’enregistrement avec le prix minimum pour chaque produit unique.

Chaque fonction utilisée est expliquée en détail dans la suite du notebook.


In [ ]:
# Importation des bibliothèques nécessaires
import os
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Définir le répertoire de base : ici nous utilisons le répertoire de travail actuel
BASE_DIR = Path.cwd()  # ou adaptez ce chemin en fonction de votre arborescence

# Définir les répertoires pour les données brutes et les données nettoyées
RAW_DATA_DIR = BASE_DIR / 'data' / 'raw' / 'ebay' / 'laptops'
CLEANED_DATA_DIR = BASE_DIR / 'data' / 'cleaned' / 'ebay' / 'laptops'

# Créer le répertoire de données nettoyées s'il n'existe pas
CLEANED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Afficher les chemins pour vérification
print(f"Base directory: {BASE_DIR}")
print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Cleaned data directory: {CLEANED_DATA_DIR}")

# Vérifier que le répertoire des données brutes existe
if not RAW_DATA_DIR.exists():
    raise FileNotFoundError(f"Le répertoire des données brutes est introuvable : {RAW_DATA_DIR}")

# Vérifier la présence de fichiers CSV dans le répertoire raw
files = list(RAW_DATA_DIR.glob('*.csv'))
if not files:
    print(f"Aucun fichier CSV trouvé dans {RAW_DATA_DIR}")
else:
    print(f"{len(files)} fichier(s) CSV trouvé(s) dans {RAW_DATA_DIR}")


### Fonction : `clean_price`

Cette fonction prend une valeur de la colonne `Price` (souvent une chaîne de caractères contenant des symboles ou des séparateurs) et :

- Supprime les caractères non numériques (en gardant le point et la virgule).
- Remplace la virgule par un point et supprime les espaces.
- Gère les cas où il y a plusieurs points en ne conservant qu’un seul point décimal.
- Convertit finalement la chaîne nettoyée en nombre flottant (`float`) ou retourne `np.nan` si la chaîne est vide.


In [ ]:
def clean_price(price):

    if isinstance(price, str):
        # Conserver uniquement les chiffres, le point et la virgule
        cleaned = ''.join(c for c in price if c.isdigit() or c in {'.', ','})
        # Remplacer la virgule par un point et supprimer les espaces éventuels
        cleaned = cleaned.replace(',', '.').replace(' ', '')
        # Gérer le cas de plusieurs points en ne gardant qu’un seul point décimal
        if '.' in cleaned:
            parts = cleaned.split('.')
            if len(parts) > 2:
                cleaned = parts[0] + '.' + ''.join(parts[1:])
        return float(cleaned) if cleaned else np.nan
    return price


### Fonction : `convert_to_gb`

Cette fonction convertit les valeurs de mémoire (RAM ou Storage) en Gigaoctets (GB).
Elle accepte une chaîne de caractères contenant un nombre suivi d’une unité (par exemple, "8GB", "1.5 TB" ou "512 MB") et :

- Extrait la valeur numérique et l’unité.
- Convertit l’unité en GB (1 TB = 1000 GB, 1 MB = 1/1000 GB).
- Retourne `np.nan` si la valeur est manquante ou invalide.


In [ ]:
def convert_to_gb(value):
    if pd.isna(value) or value in ['N/A', '', ' ']:
        return np.nan
    match = re.match(r'(\d+\.?\d*)\s*([GTM]B?|GB)', str(value), re.IGNORECASE)
    if match:
        num, unit = match.groups()
        num = float(num)
        unit = unit.upper()
        if 'T' in unit:
            return num * 1000
        elif 'M' in unit:
            return num / 1000
        return num
    return np.nan


### Fonction : `impute_missing_values`

Pour certaines colonnes critiques (comme `RAM`, `Storage`, `Screen Size`, `CPU` et `Price`), il est important de gérer les valeurs manquantes.
Cette fonction applique différentes stratégies d’imputation :

- **Médiane** : pour `RAM`, `Storage`, `Screen Size` et `Price`.
- **Forward Fill (ffill)** : pour `CPU`.

Elle retourne le DataFrame avec les valeurs manquantes imputées.


In [ ]:
def impute_missing_values(df):

    imputation_rules = {
        'RAM': 'median',
        'Storage': 'median',
        'Screen Size': 'median',
        'CPU': 'ffill',
        'Price': 'median'
    }

    for col, method in imputation_rules.items():
        if col in df.columns:
            if method == 'median':
                df[col] = df[col].fillna(df[col].median())
            elif method == 'ffill':
                df[col] = df[col].ffill()
    return df


### Fonction : `clean_title`

Les titres des annonces peuvent contenir de nombreux termes et symboles superflus (spécifications techniques répétitives, mentions inutiles, etc.).
Cette fonction utilise une expression régulière pour supprimer ces mots-clés et nettoyer le titre afin de le rendre plus lisible.


In [ ]:
def clean_title(title):

    return re.sub(
        r'(?i)\b(8GB|. |PC|Notebook|Ryzen|UHD|Graphics|DDR4|AMD|W11|Win11|Win|11|Cond|!!|LOADED|TouchBar|Mac OS|Black| i3StorageWin|Gaming|Laptop|Touchscreen|Pro|15.6|Windows|RTX|FHD|LaptopWin11|HDD| ,|French|13inch|'
        r' - | /|macOS|VENTURA|FREE|SHIPPIN|i9|13.3|inches|TURBO|"|- | , |13INCH|EXCELLENT|'
        r'REFURBISHED|NEW|MWTK2LL|Qwerty|Spanish|Keyboard|British|\d+GB|\d+TB|[\d\.]+ ?GHz| GB |'
        r'rouge|Gray|BIG SUR|WEBCAM|WIFI|BLUETOOTHGB|TB|space gray|silver|gold|touch bar|GHz|'
        r'Intel|Core|i7|th|Gen|GB|Very|RAM|i5| GB| TB|GB GB|.GHZ| CPU | GPU|-|SSD|256|512|Good|'
        r'Condition|magic keyboard|✅|🔋|grade [A-B]|warranty\.\.\.)',
        '',
        str(title)
    ).strip().replace('  ', ' ')


### Fonction : `remove_duplicates_keep_min_price`

Pour éviter la redondance, cette fonction supprime les doublons en se basant sur certaines colonnes clés (`Model`, `RAM`, `CPU`, `Brand`, `Storage`).
Avant la déduplication, la colonne `Price` est convertie en numérique.
Le DataFrame est trié par prix croissant et, pour chaque groupe de produits identiques, seule la ligne avec le prix minimum est conservée.


In [ ]:
def remove_duplicates_keep_min_price(df):

    # S'assurer que la colonne Price est numérique
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    # Définir les colonnes clés pour l'unicité du produit
    key_columns = ['Model', 'RAM', 'CPU', 'Brand', 'Storage']
    # Trier par prix croissant
    df = df.sort_values(by='Price', ascending=True)
    # Supprimer les doublons en gardant la première occurrence (celle avec le prix minimum)
    df_cleaned = df.drop_duplicates(subset=key_columns, keep='first')
    return df_cleaned


### Traitement et Nettoyage des Fichiers CSV

Dans cette section, nous parcourons tous les fichiers CSV présents dans le répertoire `RAW_DATA_DIR`.
Pour chaque fichier :

1. Le fichier est chargé dans un DataFrame.
2. Les fonctions de nettoyage et de transformation sont appliquées :
   - Nettoyage de la colonne `Price`
   - Conversion des colonnes `RAM` et `Storage` en GB
   - Extraction du nombre dans `Screen Size`
   - Imputation des valeurs manquantes
   - Remplacement de valeurs manquantes dans la colonne `GPU` par `"Unknown Graphics"`
   - Nettoyage de la colonne `Title`
   - Remplacement des valeurs vides dans `Model` par celles de `Title`
   - Arrondi de la colonne `Storage`
   - Filtrage des enregistrements avec un prix inférieur à 90
   - Suppression des doublons en gardant le prix minimum

3. Le DataFrame nettoyé est sauvegardé dans le répertoire `CLEANED_DATA_DIR`.


In [ ]:
try:
    # Parcourir tous les fichiers CSV dans le dossier des données brutes
    for file in RAW_DATA_DIR.glob('*.csv'):
        print(f"Traitement du fichier : {file.name}")

        # Charger le fichier CSV dans un DataFrame
        df = pd.read_csv(file)
        print(f"{len(df)} lignes chargées depuis {file.name}")
        print(f"Colonnes disponibles : {df.columns.tolist()}")  # Pour le débogage

        # Nettoyer la colonne Price
        df['Price'] = df['Price'].apply(clean_price)

        # Convertir les colonnes RAM et Storage en GB
        for col in ['RAM', 'Storage']:
            if col in df.columns:
                df[col] = df[col].apply(convert_to_gb).astype(float)

        # Extraire le nombre de la colonne Screen Size (si présente)
        if 'Screen Size' in df.columns:
            df['Screen Size'] = df['Screen Size'].str.extract(r'(\d+\.?\d*)')[0].astype(float)

        # Imputer les valeurs manquantes
        df = impute_missing_values(df)

        # Remplacer les valeurs manquantes dans GPU par 'Unknown Graphics'
        if 'GPU' in df.columns:
            df['GPU'] = df['GPU'].fillna('Unknown Graphics')

        # Nettoyer la colonne Title
        if 'Title' in df.columns:
            df['Title'] = df['Title'].apply(clean_title)

        # Pour la colonne Model, remplacer les valeurs vides par NaN puis les combler avec Title
        if 'Model' in df.columns and 'Title' in df.columns:
            df['Model'] = df['Model'].replace('', np.nan)
            df['Model'] = df['Model'].fillna(df['Title'])

        # Arrondir Storage
        if 'Storage' in df.columns:
            df['Storage'] = df['Storage'].round(2)

        # Filtrer les lignes avec un prix inférieur à 90
        df = df[df['Price'] >= 90]

        # Supprimer les doublons en conservant le prix minimum
        df = remove_duplicates_keep_min_price(df)

        # Sauvegarder le DataFrame nettoyé dans un nouveau fichier CSV
        output_filename = CLEANED_DATA_DIR / f"{file.stem}_cleaned.csv"
        df.to_csv(output_filename, index=False)
        print(f"Fichier nettoyé sauvegardé dans : {output_filename}")
        print(f"Forme finale du DataFrame : {df.shape}\n")
except Exception as e:
    print(f"Une erreur est survenue : {e}")


#  interprétations des résultats.

Nous avons parcouru l’ensemble des fichiers CSV présents dans le répertoire des données brutes, appliqué plusieurs opérations de nettoyage et de transformation, puis sauvegardé les fichiers nettoyés dans un nouveau répertoire.
Les fonctions définies ont permis de :

- Nettoyer et convertir les formats de données (prix, RAM, stockage, taille de l’écran).
- Gérer les valeurs manquantes.
- Nettoyer les titres et uniformiser les informations.
- Supprimer les doublons en conservant les enregistrements présentant le meilleur rapport qualité/prix.

.Voila le result apres le nettoyage



In [4]:
import pandas as pd
from pathlib import Path

# Définir le chemin vers le fichier CSV nettoyé
csv_file_path = Path(r'C:\Users\AdMin\Desktop\ecommerce_scraper\data\cleaned\ebay\laptops\cleaned_laptops_2025_01_29_scrape1.csv')

df_cleaned = pd.read_csv(csv_file_path)

# Set the option to display all rows
pd.set_option('display.max_rows', None)

# Display all rows of the DataFrame
df_cleaned.head(10)

,Title,Price,RAM,CPU,Model,Brand,GPU,Screen Size,Storage,Collection Date
0,HP BookG2 Computer .6 i3 dows 10,95.33,8.0,Intel Core i3-6100U,HP ProBook 11 G2,HP,Intel HD Graphics 520,11.6,128.0,1/29/2025 22:22
1,7'' MiniComputer QuadAndroid 12ROM Netbook For...,99.00,2.0,Integrated,7'' MiniComputer QuadAndroid 12ROM Netbook For...,G-ANICA,Unknown Graphics,10.0,512.0,1/29/2025 22:23
2,Lenovo 100W – Open BOX – Model 82HY –,99.00,4.0,AMD 3000 Series,100w Gen 3 Laptop (Lenovo) - Type 82HY,Lenovo,Unknown Graphics,11.6,64.0,1/29/2025 22:22
3,Lenovo 300e .62inCeleron dows 10,99.50,4.0,Intel Celeron N4100,Lenovo 300e 2nd Gen,Lenovo,Intel UHD Graphics 600,11.6,64.0,1/29/2025 22:23
4,dows 1010.1QuadComputer Mini Netbook (pink),99.96,16.0,Intel Celeron N,dows 1010.1QuadComputer Mini Netbook (pink),Unbranded,Unknown Graphics,10.1,512.0,1/29/2025 22:22
5,7''Computer QuadPowered by Android 12.Netbook ...,99.99,2.0,Integrated,GBOOK,NBD,Unknown Graphics,7.0,512.0,1/29/2025 22:22
6,ASUS CX1500CKANanoEdgeN4500 Military Grade Chr...,109.00,8.0,Intel Celeron N4500,CX1500CKA-WB84F,ASUS,Intel UHD Graphics,15.6,64.0,1/29/2025 22:23
7,Lenovo inkPad 14Screen 4thdows Home,115.99,12.0,Intel i5,Lenovo ThinkPad T440S,Lenovo,Intel HD Graphics 4400,14.0,256.0,1/29/2025 22:23
8,Dell Latitude Computer HDi3 dows,119.40,16.0,Intel Core i3-8145U,Dell Latitude 3310,Dell,Intel HD Graphics 520,13.3,128.0,1/29/2025 22:22
9,HP Book X360G2 .6 dows 10,119.40,8.0,Intel Core i5-7Y54,HP ProBook X360 11 G2 EE,HP,Intel UHD Graphics,11.6,128.0,1/29/2025 22:23
